In [19]:
import pandas as pd
from datetime import datetime

data = pd.read_csv('../database/merged_players_and_matches_1998_2008_data_v4.csv', index_col=[0])

In [20]:
print(len(data))

3763510


In [21]:
data.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,player_id,own_team_position_in_league,rival_team_position_in_league,score_own_team,score_rival_team,goals,asists,own_goals,minute_in,minute_out,minutes_played,rating_transfermarkt,own_team_and_rival_team_position_difference,year,born,market_value
count,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.28162e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06,3.76351e+06
mean,572313,6.95963,7.39022,1.56333,1.40083,-0.495799,-0.516345,-0.540869,9.15689,38.0809,30.0822,-0.977026,0.430591,2020.59,2000.72,1.32071e+06
std,188024,6.59863,6.92476,1.46104,1.32945,0.595688,0.552151,0.499981,24.9572,43.3322,38.6985,0.319593,7.61334,2.11682,2.02015,6.30353e+06
min,34657,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-136,2011,1998,10000
25%,420460,2,2,0,0,-1,-1,-1,-1,-1,0,-1,-4,2019,1999,50000
50%,542654,6,7,1,1,-1,-1,-1,-1,-1,0,-1,0,2021,2000,150000
75%,691782,12,12,2,2,0,0,0,0,90,78,-1,5,2022,2002,400000
max,1.19336e+06,149,155,32,18,9,7,2,120,194,135,6,145,2024,2008,1.8e+08


In [22]:
print(data.groupby('country_name')['player_id'].nunique())
print('Number of unique players:', data['player_id'].nunique())

country_name
Austria         451
Azerbaijan       91
Belgium         511
Bulgaria        621
Croatia         715
Cyprus          157
Denmark         526
England        1173
France         1161
Germany        2178
Greece          515
Hungary         425
Isreal          407
Italy          3290
Moldavia        165
Netherlands     683
Norway          542
Poland          616
Portugal        546
Romania         975
Russia         1662
Scotland        274
Slovakia        443
Spain          2473
Sweden          554
Switzerland     600
Turkey         1538
Ukraine         498
Name: player_id, dtype: int64
Number of unique players: 23790


In [23]:
import numpy as np

numerical_columns = ['own_team_position_in_league', 'rival_team_position_in_league', 'score_own_team', 
                     'score_rival_team', 'minute_in', 'minute_out', 'minutes_played', 'rating_transfermarkt',
                     'goals', 'asists', 'own_goals', 'own_team_and_rival_team_position_difference', 'market_value']
categorical_columns = ['starting_position']

# Update the list of columns based on the actual columns in the dataset
numerical_columns = [col for col in numerical_columns if col in data.columns]
categorical_columns = [col for col in categorical_columns if col in data.columns]

# Replace '-1' and 'N/A' with NaN for the remaining columns
for col in numerical_columns:
    data[col] = data[col].replace(-1, np.nan)

for col in categorical_columns:
    data[col] = data[col].replace('N/A', np.nan)

In [24]:
# Current year for age calculation
current_year = datetime.now().year

# Adding age and number of years playing
data['age'] = current_year - data['born']
data['years_playing'] = current_year - data['year']

# Calculate sum of goals, assists, and own goals
data['total_goals'] = data['goals'].groupby(data['player_id']).transform('sum')
data['total_assists'] = data['asists'].groupby(data['player_id']).transform('sum')
data['total_own_goals'] = data['own_goals'].groupby(data['player_id']).transform('sum')

# Dropping rows where match was canceled
data_filtered = data[data['match_canceled'] == False]

# Calculate average goals, assists, and own goals per game played
data_filtered['avg_goals_per_game'] = data_filtered['total_goals'] / data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['avg_assists_per_game'] = data_filtered['total_assists'] / data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['avg_own_goals_per_game'] = data_filtered['total_own_goals'] / data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')

# Average score of own and rival teams
data_filtered['avg_score_own_team'] = data_filtered['score_own_team'].groupby(data_filtered['player_id']).transform('mean')
data_filtered['avg_score_rival_team'] = data_filtered['score_rival_team'].groupby(data_filtered['player_id']).transform('mean')

# Average score of own and rival teams if played
data_filtered_played = data_filtered[data_filtered['played']]
data_filtered['avg_score_own_team_if_played'] = data_filtered_played['score_own_team'].groupby(data_filtered_played['player_id']).transform('mean')
data_filtered['avg_score_rival_team_if_played'] = data_filtered_played['score_rival_team'].groupby(data_filtered_played['player_id']).transform('mean')

# Count of match results
data_filtered['count_win'] = (data_filtered['result'] == 'MatchResult.WIN').groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_draw'] = (data_filtered['result'] == 'MatchResult.DRAW').groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_loss'] = (data_filtered['result'] == 'MatchResult.LOSS').groupby(data_filtered['player_id']).transform('sum')

# Count of match types
data_filtered['count_cup_matches'] = (data_filtered['match_type'] == 'cup').groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_league_matches'] = (data_filtered['match_type'] == 'league').groupby(data_filtered['player_id']).transform('sum')

# Number of different rival teams played against
data_filtered['different_rival_teams'] = data_filtered.groupby('player_id')['rival_team_name'].transform('nunique')

# Number of different own teams played for
data_filtered['different_own_teams'] = data_filtered.groupby('player_id')['own_team_name'].transform('nunique')

# Number of suspensions for red cards and yellow cards
data_filtered['suspensions_red_card'] = data_filtered['suspended_for_red_card'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['suspensions_yellow_cards'] = data_filtered['suspended_for_yellow_cards'].groupby(data_filtered['player_id']).transform('sum')

# Count of various match participation statuses
data_filtered['count_starting_lineup'] = data_filtered['in_starting_lineup'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_stayed_on_bench'] = data_filtered['stayed_on_bench'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_played'] = data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')
data_filtered['count_skipped_injury'] = data_filtered['skipped_cause_of_injury'].groupby(data_filtered['player_id']).transform('sum')

# Count of matches as a captain
data_filtered['matches_as_captain'] = data_filtered['captain'].groupby(data_filtered['player_id']).transform('sum')

# Average and total minutes played
data_filtered['avg_minutes_played'] = data_filtered['minutes_played'].groupby(data_filtered['player_id']).transform('mean')
data_filtered['total_minutes_played'] = data_filtered['minutes_played'].groupby(data_filtered['player_id']).transform('sum')

# Average minute in and out
data_filtered['avg_minute_in'] = data_filtered['minute_in'].replace('-1', np.nan).astype(float).groupby(data_filtered['player_id']).transform('mean')
data_filtered['avg_minute_out'] = data_filtered['minute_out'].replace('-1', np.nan).astype(float).groupby(data_filtered['player_id']).transform('mean')

# Number of times out because of injury (broken in scraping deleted)
# data_filtered['times_out_injury'] = (data_filtered['out_cause_of_injury'] == 1).groupby(data_filtered['player_id']).transform('sum')

# Alternative approach for calculating the most frequent position
def most_frequent_position(grouped_df):
    return grouped_df['starting_position'].mode().iloc[0] if not grouped_df['starting_position'].mode().empty else 'N/A'

most_frequent_positions = data_filtered.groupby('player_id').apply(most_frequent_position)
data_filtered = data_filtered.merge(most_frequent_positions.rename('most_frequent_position'), on='player_id', how='left')

/var/folders/lq/yv936gk54wg_p3fy9yv8dz240000gn/T/ipykernel_74629/780344994.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['avg_goals_per_game'] = data_filtered['total_goals'] / data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')
/var/folders/lq/yv936gk54wg_p3fy9yv8dz240000gn/T/ipykernel_74629/780344994.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['avg_assists_per_game'] = data_filtered['total_assists'] / data_filtered['played'].groupby(data_

/var/folders/lq/yv936gk54wg_p3fy9yv8dz240000gn/T/ipykernel_74629/780344994.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['count_played'] = data_filtered['played'].groupby(data_filtered['player_id']).transform('sum')
/var/folders/lq/yv936gk54wg_p3fy9yv8dz240000gn/T/ipykernel_74629/780344994.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['count_skipped_injury'] = data_filtered['skipped_cause_of_injury'].groupby(data_filtered['player_id']).transform('sum')
/var/folders/

In [25]:
# Drop unnecessary columns
columns_to_drop = ['round', 'date', 'own_team_name', 'rival_team_name', 'own_team_position_in_league', 
                   'rival_team_position_in_league','match_type','match_canceled','score_own_team', 
                   'score_rival_team', 'result', 'match_url', 'suspended_for_red_card', 'suspended_for_yellow_cards', 
                   'received_red_cards', 'minute_in', 'minute_out', 'minutes_played', 'out_cause_of_injury','rating_transfermarkt',
                   'rating_transfermarkt', 'own_team_and_rival_team_position_difference', 'year', 'in_starting_lineup',
                   'stayed_on_bench', 'played', 'skipped_cause_of_injury', 'skipped_cause_of_injury', 'starting_position',
                   'captain', 'goals','asists', 'own_goals', 'received_yellow_card', 'received_two_yellows_cards'
                  ]
data_filtered.drop(columns_to_drop, axis=1, inplace=True)

In [26]:
# Aggregating data to have one row per player, taking the first value in each column
agg_funcs_first_only = {col: 'first' for col in data_filtered.columns if col != 'player_id'}
player_data_aggregated_first_only = data_filtered.groupby('player_id').agg(agg_funcs_first_only)

# Reset index to make player_id a column again
player_data_aggregated_first_only.reset_index(inplace=True)

In [27]:
player_data_aggregated_first_only.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,player_id,born,market_value,age,years_playing,total_goals,total_assists,total_own_goals,avg_goals_per_game,avg_assists_per_game,...,suspensions_yellow_cards,count_starting_lineup,count_stayed_on_bench,count_played,count_skipped_injury,matches_as_captain,avg_minutes_played,total_minutes_played,avg_minute_in,avg_minute_out
count,23790,23790,23790,23790,23790,23790,23790,23790,23265,23265,...,23790,23790,23790,23790,23790,23790,23790,23790,23204,23265
mean,677852,2001.45,702373,22.5451,5.17024,7.26019,4.00979,0.130223,0.0818924,0.0420517,...,0.356074,23.2518,23.2518,72.4871,4.95565,1.96822,28.2573,4757.97,31.2471,84.5777
std,217233,2.15254,4.34488e+06,2.15254,2.35427,12.1593,6.72831,0.417701,0.103474,0.0552997,...,0.848791,22.5937,22.5937,57.5321,12.781,6.5651,15.9328,4095.66,22.1548,4.37426
min,34657,1998,10000,16,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
25%,510439,2000,25000,21,3,0,0,0,0,0,...,0,8,8,28,0,0,17.348,1583.25,13.1075,82.191
50%,656131,2001,75000,23,5,3,1,0,0.047619,0.0232558,...,0,17,17,61,0,0,27.7298,3870,28.4138,84.6667
75%,837251,2003,200000,24,7,9,5,0,0.117647,0.0666667,...,0,31,31,104,3,0,38.2273,6851,46.75,87.349
max,1.19336e+06,2008,1.8e+08,26,13,265,118,5,2,1,...,15,378,378,504,200,164,120,40514,118,120


In [28]:
player_data_aggregated_first_only.to_csv('aggregated_players_and_matches_data_v4.csv')

In [29]:
import sweetviz as sv

player_data_aggregated_first_only.drop(['url', 'born', 'player_id', 'player_name'], axis=1, inplace=True)
player_data_aggregated_first_only.drop(['country_name', 'most_frequent_position'], axis=1, inplace=True)
# Checking for null cells
# print(players_and_matches_df.isnull().sum())

player_data_aggregated_first_only.dropna(inplace=True)

In [30]:
my_report = sv.analyze(player_data_aggregated_first_only, target_feat='market_value')

Done! Use 'show' commands to display/save.   |██| [100%]   00:01 -> (00:00 left)


In [31]:
my_report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
